In [117]:
import mne
import glob
import pandas as pd
from pathlib import Path
import numpy as np

In [118]:
def pairwise(iterable):
    "s -> (s0, s1), (s2, s3), (s4, s5), ..."
    a = iter(iterable)
    return zip(a, a)

def set_labels(cropped,raw,labels,score_type='valence',clip_duration=50):
    anon = {i:[] for i in ['onsets','duration','description']}
    for annotation in pairwise(raw.annotations):
        video_name =annotation[0]['description'].split('_')
        if video_name[0]=='video':
            if cropped:
                onset = annotation[1]['onset']-clip_duration
                duration = clip_duration
            else:
                onset = annotation[0]['onset']
                duration = annotation[1]['onset']-annotation[0]['onset']
            video_id = int(video_name[2])
            idx = labels.index[labels['video_id'] == video_id]
            anon['onsets'].append(onset)
            anon['duration'].append(duration)
            anon['description'].append(labels[score_type].values[idx][0])
    annotations = mne.Annotations(
        onset=anon['onsets'],  # in seconds, the moment each epoch starts
        duration=anon['duration'],  # in seconds, too, duration of each epoch (if data clipped 50s, should be equal for all)
        description=anon['description'],
    )
    raw.set_annotations(annotations);
    return raw,annotations.description


def get_segments(raw,labels,score_type,cropped):
    raw,labels = set_labels(cropped,raw,labels,score_type)
    segments = []
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    start_idx = events_from_annot[:,0]
    end_idx = events_from_annot[:,0] + (raw.annotations.duration*raw.info['sfreq']).astype(int)
    for indices in zip(start_idx,end_idx):
        segments.append(raw.get_data()[:,indices[0]:indices[1]])
    return segments

def get_segments_labels(folder,subject,cropped= True, score_type='valence'):
    raw = mne.io.Raw(folder+'Data/'+subject+'_eeg.fif')
    labels = pd.read_csv(folder+'ratings/'+subject+'.csv')
    segments = get_segments(raw,labels,score_type,cropped)
    return segments, labels

In [119]:
cropped = False
folder = "/home/s210382/Insai/DTU/Data/Nexus/"
files = sorted(glob.glob(folder+"Data/*"))
for filename in files[0:1]:
    subject = Path(filename).stem[:-4]
    print('Subject:',subject)
    segments, labels = get_segments_labels(folder,subject,cropped,'valence')
    ##TODO: save segments and labels in a seperate file
    

Subject: kaleem
Opening raw data file /home/s210382/Insai/DTU/Data/Nexus/Data/kaleem_eeg.fif...
    Range : 0 ... 912219 =      0.000 ...  3648.876 secs
Ready.
Used Annotations descriptions: ['1', '2', '3', '4', '5']
Subject: subj10
Opening raw data file /home/s210382/Insai/DTU/Data/Nexus/Data/subj10_eeg.fif...
    Range : 0 ... 849419 =      0.000 ...  3397.676 secs
Ready.
Used Annotations descriptions: ['1', '2', '4', '5']
Subject: subj13
Opening raw data file /home/s210382/Insai/DTU/Data/Nexus/Data/subj13_eeg.fif...
    Range : 0 ... 904669 =      0.000 ...  3618.676 secs
Ready.
Used Annotations descriptions: ['1', '2', '3', '4', '5']
Subject: subj14
Opening raw data file /home/s210382/Insai/DTU/Data/Nexus/Data/subj14_eeg.fif...
    Range : 0 ... 825369 =      0.000 ...  3322.205 secs
Ready.
Used Annotations descriptions: ['1', '2', '3', '4', '5']
